In [1]:
#import numpy to use seed 
import numpy as np
import pandas as pd
from numpy.random import seed
np.random.seed(1)

In [2]:
#import other dependencies

import os
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

Read CSV create, and clean dataframe

In [3]:
#Read in CSV and create dataframe
mlb_teams= pd.read_csv("Teams.csv")
mlb_teams = mlb_teams.dropna()
mlb_teams

,yearID,lgID,G,G.1,W,L,WPCT,WSWin,R,AB,...,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name
0,2004.0,AL,162.0,81.0,92.0,70.0,0.36,N,836.0,5675.0,...,37.0,162.0,450.0,942.0,143.0,4.28,2.0,11.0,50.0,Anaheim Angels
1,2003.0,AL,162.0,82.0,77.0,85.0,0.48,N,736.0,5487.0,...,33.0,150.0,476.0,838.0,129.0,4.28,5.0,9.0,39.0,Anaheim Angels
2,2002.0,AL,162.0,81.0,99.0,63.0,0.54,Y,851.0,5678.0,...,32.0,152.0,462.0,805.0,117.0,3.69,7.0,14.0,54.0,Anaheim Angels
3,2001.0,AL,162.0,81.0,75.0,87.0,0.48,N,691.0,5551.0,...,26.0,158.0,494.0,1001.0,116.0,4.20,6.0,1.0,43.0,Anaheim Angels
4,2000.0,AL,162.0,81.0,82.0,80.0,0.46,N,864.0,5628.0,...,34.0,236.0,608.0,1024.0,93.0,5.00,5.0,3.0,46.0,Anaheim Angels
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2006.0,NL,162.0,81.0,71.0,91.0,0.50,N,746.0,5495.0,...,22.0,164.0,594.0,1156.0,123.0,5.03,1.0,3.0,32.0,Washington Nationals
1225,2005.0,NL,162.0,81.0,81.0,81.0,0.51,N,639.0,5426.0,...,32.0,117.0,491.0,1090.0,45.0,3.87,4.0,9.0,51.0,Washington Nationals
1226,1971.0,AL,159.0,81.0,63.0,96.0,0.43,N,537.0,5290.0,...,30.0,86.0,575.0,956.0,68.0,3.70,30.0,10.0,26.0,Washington Senators
1227,1970.0,AL,162.0,81.0,70.0,92.0,0.46,N,626.0,5460.0,...,28.0,138.0,635.0,989.0,72.0,3.80,20.0,11.0,40.0,Washington Senators


In [4]:
#Get list of dataframe columns

mlb_teams.columns

Index(['yearID', 'lgID', 'G', 'G.1', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H',
       '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name'],
      dtype='object')

In [5]:
#Select columns for dataframe


mlb_df = mlb_teams[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]
mlb_df.head(20)

,yearID,W,L,WPCT,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name
0,2004.0,92.0,70.0,0.36,N,836.0,5675.0,1603.0,272.0,37.0,162.0,450.0,942.0,143.0,4.28,2.0,11.0,50.0,Anaheim Angels
1,2003.0,77.0,85.0,0.48,N,736.0,5487.0,1473.0,276.0,33.0,150.0,476.0,838.0,129.0,4.28,5.0,9.0,39.0,Anaheim Angels
2,2002.0,99.0,63.0,0.54,Y,851.0,5678.0,1603.0,333.0,32.0,152.0,462.0,805.0,117.0,3.69,7.0,14.0,54.0,Anaheim Angels
3,2001.0,75.0,87.0,0.48,N,691.0,5551.0,1447.0,275.0,26.0,158.0,494.0,1001.0,116.0,4.20,6.0,1.0,43.0,Anaheim Angels
4,2000.0,82.0,80.0,0.46,N,864.0,5628.0,1574.0,309.0,34.0,236.0,608.0,1024.0,93.0,5.00,5.0,3.0,46.0,Anaheim Angels
5,1999.0,70.0,92.0,0.53,N,711.0,5494.0,1404.0,248.0,22.0,158.0,511.0,1022.0,71.0,4.79,4.0,7.0,37.0,Anaheim Angels
6,1998.0,85.0,77.0,0.46,N,787.0,5630.0,1530.0,314.0,27.0,147.0,510.0,1028.0,93.0,4.49,3.0,5.0,52.0,Anaheim Angels
7,1997.0,84.0,78.0,0.45,N,829.0,5628.0,1531.0,279.0,25.0,161.0,617.0,953.0,126.0,4.52,9.0,5.0,39.0,Anaheim Angels
8,2014.0,64.0,98.0,0.57,N,615.0,5552.0,1379.0,259.0,47.0,118.0,398.0,1165.0,86.0,4.26,2.0,4.0,35.0,Arizona Diamondbacks
9,2013.0,81.0,81.0,0.47,N,685.0,5676.0,1468.0,302.0,31.0,130.0,519.0,1142.0,62.0,3.92,6.0,7.0,38.0,Arizona Diamondbacks


In [6]:
#Use getdummies to convert strings

data_LG = mlb_df.copy()
data_bin = pd.get_dummies(data_LG, columns=["WSWin"])
data_bin = data_bin.drop(columns=['WSWin_N'])

data_bin.head()

,yearID,W,L,WPCT,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name,WSWin_Y
0,2004.0,92.0,70.0,0.36,836.0,5675.0,1603.0,272.0,37.0,162.0,450.0,942.0,143.0,4.28,2.0,11.0,50.0,Anaheim Angels,0
1,2003.0,77.0,85.0,0.48,736.0,5487.0,1473.0,276.0,33.0,150.0,476.0,838.0,129.0,4.28,5.0,9.0,39.0,Anaheim Angels,0
2,2002.0,99.0,63.0,0.54,851.0,5678.0,1603.0,333.0,32.0,152.0,462.0,805.0,117.0,3.69,7.0,14.0,54.0,Anaheim Angels,1
3,2001.0,75.0,87.0,0.48,691.0,5551.0,1447.0,275.0,26.0,158.0,494.0,1001.0,116.0,4.20,6.0,1.0,43.0,Anaheim Angels,0
4,2000.0,82.0,80.0,0.46,864.0,5628.0,1574.0,309.0,34.0,236.0,608.0,1024.0,93.0,5.00,5.0,3.0,46.0,Anaheim Angels,0


In [7]:
#Set X(data) and y(team)variables

data = data_bin.drop("name", axis=1)
outcome = data_bin["name"]
print(data.shape, outcome.shape)

(1229, 18) (1229,)


In [8]:
outcome.unique()

array(['Anaheim Angels', 'Arizona Diamondbacks', 'Atlanta Braves',
       'Baltimore Orioles', 'Boston Red Sox', 'California Angels',
       'Chicago Cubs', 'Chicago White Sox', 'Cincinnati Reds',
       'Cleveland Indians', 'Colorado Rockies', 'Detroit Tigers',
       'Florida Marlins', 'Houston Astros', 'Kansas City Royals',
       'Los Angeles Angels of Anaheim', 'Los Angeles Dodgers',
       'Miami Marlins', 'Milwaukee Brewers', 'Minnesota Twins',
       'Montreal Expos', 'New York Mets', 'New York Yankees',
       'Oakland Athletics', 'Philadelphia Phillies', 'Pittsburgh Pirates',
       'San Diego Padres', 'San Francisco Giants', 'Seattle Mariners',
       'St. Louis Cardinals', 'Tampa Bay Devil Rays', 'Tampa Bay Rays',
       'Texas Rangers', 'Toronto Blue Jays', 'Washington Nationals',
       'Washington Senators'], dtype=object)

Preprocess data

In [9]:
#Import SciKit libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical

In [10]:
#Split data and teams into training and test data

X_train, X_test, y_train, y_test = train_test_split(data, outcome, train_size=0.6, test_size=0.4, random_state=42)
X_train

,yearID,W,L,WPCT,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,WSWin_Y
714,1977.0,75.0,87.0,0.54,665.0,5675.0,1474.0,294.0,50.0,138.0,478.0,877.0,88.0,4.01,31.0,11.0,33.0,0
643,2014.0,70.0,92.0,0.54,715.0,5567.0,1412.0,316.0,27.0,128.0,544.0,1329.0,99.0,4.57,2.0,7.0,38.0,0
845,1981.0,64.0,45.0,0.44,458.0,3677.0,910.0,119.0,26.0,104.0,342.0,647.0,98.0,3.30,60.0,11.0,10.0,0
796,1985.0,97.0,64.0,0.48,839.0,5458.0,1458.0,272.0,31.0,176.0,620.0,771.0,155.0,3.69,25.0,9.0,49.0,0
722,1969.0,52.0,110.0,0.47,582.0,5419.0,1300.0,202.0,33.0,125.0,529.0,962.0,52.0,4.33,26.0,8.0,21.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,2008.0,61.0,101.0,0.41,671.0,5643.0,1498.0,285.0,20.0,124.0,417.0,890.0,90.0,4.73,4.0,4.0,36.0,0
1095,1993.0,87.0,75.0,0.52,758.0,5551.0,1508.0,262.0,34.0,118.0,588.0,882.0,153.0,4.09,5.0,7.0,54.0,0
1130,2014.0,77.0,85.0,0.53,612.0,5516.0,1361.0,263.0,24.0,117.0,527.0,1124.0,63.0,3.56,3.0,22.0,37.0,0
860,2012.0,81.0,81.0,0.55,684.0,5544.0,1414.0,271.0,28.0,158.0,454.0,1094.0,116.0,3.83,5.0,11.0,42.0,0


In [11]:
#Scale data using StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_test

548     Los Angeles Angels of Anaheim
1125             Tampa Bay Devil Rays
244                 Chicago White Sox
552               Los Angeles Dodgers
1163                    Texas Rangers
                    ...              
97                  Baltimore Orioles
1053                 Seattle Mariners
1148                    Texas Rangers
962                  San Diego Padres
631                 Milwaukee Brewers
Name: name, Length: 492, dtype: object

In [12]:
#Label-encode team

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
encoded_y_train

array([20, 19, 23, 22, 20,  2, 19, 28, 17, 29, 35,  8,  7, 27, 26,  8, 34,
        6, 13, 11,  7, 24, 33,  8, 22, 26,  7, 22,  3,  3, 24,  2, 22, 12,
       12, 24, 18, 20,  3,  3, 16,  3, 13, 27, 12, 18, 25, 11, 28, 22, 32,
        3, 16, 28,  2,  8, 20,  7,  6,  4, 32, 10,  4, 23,  7, 33,  1, 12,
       25, 11, 30, 26, 16, 13, 31,  4, 14, 31, 14,  5, 33, 21, 18, 34,  0,
       16, 24,  9,  9, 21, 24,  7, 25, 25, 23, 18, 24, 14, 24, 26, 33, 32,
       12, 14, 27,  6, 28,  6,  3,  4, 18, 29, 23,  4, 23, 11, 27, 14, 25,
       14, 20,  9, 10,  0,  9, 32, 13, 23, 25,  5, 12,  4, 19, 14,  3, 33,
        5, 23, 11,  7,  2,  4,  8, 20,  9, 33, 14, 25, 26, 13,  9, 27,  5,
        2, 14,  6, 19, 23,  5, 25,  5, 14, 26, 21, 13,  3, 27, 16, 19,  7,
        6, 27,  4, 24, 34, 26,  5,  2,  3, 27, 16, 19, 28, 33, 13, 13,  1,
       20, 20, 25, 28,  9, 24, 29, 10, 13,  7, 27,  7, 12, 11, 22,  5,  1,
       23, 24, 16,  2,  7, 32,  8, 24, 20, 10, 25, 24, 25, 21, 23, 15, 28,
       20, 26, 13, 11, 19

In [13]:
#One hot_encode the team data

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
for x in y_test_categorical:
    print(x)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 

 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 

 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 

 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 

Create Neural Network Model

In [14]:
# Create model and add layers
model = Sequential()
model.add(Dense(100, input_dim=18, activation='relu'))

model.add(Dense(36, activation='softmax'))

In [15]:
#View model summary

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1900      
_________________________________________________________________
dense_1 (Dense)              (None, 36)                3636      
Total params: 5,536
Trainable params: 5,536
Non-trainable params: 0
_________________________________________________________________


Train Model

In [16]:
#Compile model
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
print(outcome.shape)

(1229,)


In [18]:
#Fit data to train model

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
24/24 - 0s - loss: 0.1274 - accuracy: 0.0339
Epoch 2/100
24/24 - 0s - loss: 0.1224 - accuracy: 0.0814
Epoch 3/100
24/24 - 0s - loss: 0.1189 - accuracy: 0.1058
Epoch 4/100
24/24 - 0s - loss: 0.1162 - accuracy: 0.1275
Epoch 5/100
24/24 - 0s - loss: 0.1138 - accuracy: 0.1533
Epoch 6/100
24/24 - 0s - loss: 0.1116 - accuracy: 0.1560
Epoch 7/100
24/24 - 0s - loss: 0.1098 - accuracy: 0.1723
Epoch 8/100
24/24 - 0s - loss: 0.1080 - accuracy: 0.1967
Epoch 9/100
24/24 - 0s - loss: 0.1063 - accuracy: 0.2198
Epoch 10/100
24/24 - 0s - loss: 0.1048 - accuracy: 0.2225
Epoch 11/100
24/24 - 0s - loss: 0.1033 - accuracy: 0.2402
Epoch 12/100
24/24 - 0s - loss: 0.1019 - accuracy: 0.2415
Epoch 13/100
24/24 - 0s - loss: 0.1006 - accuracy: 0.2497
Epoch 14/100
24/24 - 0s - loss: 0.0993 - accuracy: 0.2700
Epoch 15/100
24/24 - 0s - loss: 0.0981 - accuracy: 0.2754
Epoch 16/100
24/24 - 0s - loss: 0.0971 - accuracy: 0.2754
Epoch 17/100
24/24 - 0s - loss: 0.0960 - accuracy: 0.2877
Epoch 18/100
24/24 - 0s

Evaluate Model

In [33]:
#Evaluate model

model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: in user code:

    C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\keras\engine\training.py:1224 test_function  *
        return step_function(self, iterator)
    C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\keras\engine\training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\swill\Anaconda

In [21]:
#Preprocess and predict 2015 MLB WS Champ
mlb_teams2015= pd.read_csv("2015.csv")

mlb2015_df = mlb_teams2015[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]


data2015_LG = mlb2015_df.copy()
data2015_bin = pd.get_dummies(data2015_LG, columns=["WSWin"])
data2015_bin = data2015_bin.drop(columns=['WSWin_N'])

feature2015 = data2015_bin.drop("name", axis=1)
team2015 = data2015_bin["name"]


X_feature2015_scaler = StandardScaler().fit(feature2015)
X_feature2015_scaled = X_feature2015_scaler.transform(feature2015)

label2015_encoder = LabelEncoder()
label2015_encoder.fit(team2015)
encoded_y_2015 = label2015_encoder.transform(team2015)

y_2015_categorical = to_categorical(encoded_y_2015)

encoded_predictions2015 = model.predict_classes(X_feature2015_scaled[:10])
prediction2015_labels = label2015_encoder.inverse_transform(encoded_predictions2015)

print(f"Predicted classes: {prediction2015_labels}")


Predicted classes: ['Florida Marlins' 'Minnesota Twins' 'Chicago Cubs' 'Milwaukee Brewers'
 'Cincinnati Reds' 'Milwaukee Brewers' 'Tampa Bay Devil Rays'
 'Cleveland Indians' 'Colorado Rockies' 'Milwaukee Brewers']


In [22]:
#Preprocess and predict 2016 MLB WS Champ
mlb_teams2016= pd.read_csv("2016.csv")

mlb2016_df = mlb_teams2016[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2016_LG = mlb2016_df.copy()
data2016_bin = pd.get_dummies(data2016_LG, columns=["WSWin"])
data2016_bin = data2016_bin.drop(columns=['WSWin_N'])

feature2016 = data2016_bin.drop("name", axis=1)
team2016 = data2016_bin["name"]

X_feature2016_scaler = StandardScaler().fit(feature2016)
X_feature2016_scaled = X_feature2016_scaler.transform(feature2016)

label2016_encoder = LabelEncoder()
label2016_encoder.fit(team2016)
encoded_y_2016 = label2015_encoder.transform(team2016)
encoded_y_2016
y_2016_categorical = to_categorical(encoded_y_2016)


encoded_predictions2016 = model.predict_classes(X_feature2016_scaled[:10])
prediction2016_labels = label2016_encoder.inverse_transform(encoded_predictions2016)

print(f"Predicted classes: {prediction2016_labels}")

Predicted classes: ['Tampa Bay Devil Rays' 'Minnesota Twins' 'Chicago Cubs'
 'Cleveland Indians' 'Boston Red Sox' 'Milwaukee Brewers'
 'Tampa Bay Devil Rays' 'Philadelphia Phillies' 'Colorado Rockies'
 'Milwaukee Brewers']


In [23]:
#Preprocess and predict 2017 MLB WS Champ
mlb_teams2017= pd.read_csv("2017.csv")

mlb2017_df = mlb_teams2017[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2017_LG = mlb2017_df.copy()
data2017_bin = pd.get_dummies(data2017_LG, columns=["WSWin"])
data2017_bin = data2017_bin.drop(columns=['WSWin_N',])

feature2017 = data2017_bin.drop("name", axis=1)
team2017 = data2017_bin["name"]

X_feature2017_scaler = StandardScaler().fit(feature2017)
X_feature2017_scaled = X_feature2017_scaler.transform(feature2017)

label2017_encoder = LabelEncoder()
label2017_encoder.fit(team2017)
encoded_y_2017 = label2017_encoder.transform(team2017)

y_2017_categorical = to_categorical(encoded_y_2017)




encoded_predictions2017 = model.predict_classes(X_feature2017_scaled[:10])
prediction2017_labels = label2017_encoder.inverse_transform(encoded_predictions2017)

print(f"Predicted classes: {prediction2017_labels}")

Predicted classes: ['Philadelphia Phillies' 'Minnesota Twins' 'Toronto Blue Jays'
 'New York Yankees' 'Philadelphia Phillies' 'Colorado Rockies'
 'Milwaukee Brewers' 'Boston Red Sox' 'Philadelphia Phillies'
 'Detroit Tigers']


In [24]:
#Preprocess and predict 2018 MLB WS Champ
mlb_teams2018= pd.read_csv("2018.csv")

mlb2018_df = mlb_teams2018[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2018_LG = mlb2018_df.copy()
data2018_bin = pd.get_dummies(data2018_LG, columns=["WSWin"])
data2018_bin = data2018_bin.drop(columns=['WSWin_N',])

feature2018 = data2018_bin.drop("name", axis=1)
team2018 = data2018_bin["name"]

X_feature2018_scaler = StandardScaler().fit(feature2018)
X_feature2018_scaled = X_feature2018_scaler.transform(feature2018)

label2018_encoder = LabelEncoder()
label2018_encoder.fit(team2018)
encoded_y_2018 = label2018_encoder.transform(team2018)

y_2018_categorical = to_categorical(encoded_y_2018)




encoded_predictions2018 = model.predict_classes(X_feature2018_scaled[:10])
prediction2018_labels = label2018_encoder.inverse_transform(encoded_predictions2018)

print(f"Predicted classes: {prediction2018_labels}")

Predicted classes: ['Philadelphia Phillies' 'Philadelphia Phillies' 'Seattle Mariners'
 'Cincinnati Reds' 'Toronto Blue Jays' 'Tampa Bay Devil Rays'
 'Milwaukee Brewers' 'Milwaukee Brewers' 'Philadelphia Phillies'
 'Florida Marlins']


In [25]:
#Preprocess and predict 2019 MLB WS Champ
mlb2019_teams= pd.read_csv("MLB 2019 season.csv")



mlb2019_df = mlb2019_teams[['yearID','W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]


data2019_LG = mlb2019_df.copy()
data2019_bin = pd.get_dummies(data2019_LG, columns=["WSWin"])
data2019_bin = data2019_bin.drop(columns=['WSWin_N'])



feature2019 = data2019_bin.drop("name", axis=1)
team2019 = data2019_bin["name"]


X_feature2019_scaler = StandardScaler().fit(feature2019)
X_feature2019_scaled = X_feature2019_scaler.transform(feature2019)


label_team_encoder2019 = LabelEncoder()
label_team_encoder2019.fit(team2019)
encoded_y_team2019 = label_team_encoder2019.transform(team2019)


y_team2019_categorical = to_categorical(encoded_y_team2019)

encoded_predictions2019 = model.predict_classes(feature2019)
prediction2019_labels = label_team_encoder2019.inverse_transform(encoded_predictions2019)

print(f"Predicted classes: {prediction2019_labels}")


Predicted classes: ['Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers'
 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers'
 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers'
 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers'
 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers'
 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers'
 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers' 'Detroit Tigers'
 'Detroit Tigers' 'Detroit Tigers' 'Chicago White Sox' 'Chicago White Sox'
 'Chicago White Sox' 'Chicago White Sox' 'Chicago White Sox']


In [32]:
#Preprocess and predict 2020 MLB WS Champ
mlb2020_teams= pd.read_csv("2020.csv")

mlb2020_df = mlb2020_teams[['yearID','W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2020_LG = mlb2020_df.copy()
data2020_bin = pd.get_dummies(data2020_LG, columns=["WSWin"])
data2020_bin = data2020_bin.drop(columns=['WSWin_Y'])

feature2020 = data2020_bin.drop("name", axis=1)
team2020 = data2020_bin["name"]



X_feature2020_scaler = StandardScaler().fit(feature2020)
X_feature2020_scaled = X_feature2020_scaler.transform(feature2020)


label_team_encoder2020 = LabelEncoder()
label_team_encoder2020.fit(team2020)
encoded_y_team2020 = label_team_encoder2020.transform(team2020)


y_team2020_categorical = to_categorical(encoded_y_team2020)

encoded_predictions2020 = model.predict_classes(X_feature2020_scaled[:10])
prediction2020_labels = label2018_encoder.inverse_transform(encoded_predictions2020)

print(f"Predicted classes: {prediction2020_labels}")


Predicted classes: ['Chicago Cubs' 'Boston Red Sox' 'Chicago Cubs' 'Toronto Blue Jays'
 'Chicago Cubs' 'Toronto Blue Jays' 'Baltimore Orioles' 'New York Mets'
 'Philadelphia Phillies' 'Milwaukee Brewers']


Predict World Series Champs

In [34]:
model.save("smartphone_trained.h5")

TypeError: can't pickle _thread.RLock objects